In [2]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


In [54]:
#spark.read.csv() 메소드를 이용하여 csv 파일을 로드하고 DataFrame으로 변환. 
# pandas_df = pd.read_csv('/FileStore/tables/titanic_train.csv', header='infer')
df=spark.read.csv('C:\\Users\\jihun\\Downloads\\stats (7).csv',header=True,inferSchema=True)


from pyspark.sql.types import StructType,StructField, StringType
import re

for each in df.schema.names:
    df = df.withColumnRenamed(each,  re.sub(r'\s+([a-zA-Z_][a-zA-Z_0-9]*)\s*','',each.replace(' ', '')))
# 일부 컬럼에 있는 공백 없애기    
    
# pandas DataFrame을 spark DataFrame으로 부터 생성. 
pdf = df.select('*').toPandas()


In [55]:
df.cache()

DataFrame[last_name: string, first_name: string, player_id: int, year: int, p_k_percent: double, p_bb_percent: double, xwoba: double, z_swing_percent: double, z_swing_miss_percent: double, oz_swing_percent: double, oz_swing_miss_percent: double, oz_contact_percent: double, out_zone_percent: double, iz_contact_percent: double, in_zone_percent: double, edge_percent: double, whiff_percent: double, fastball_avg_speed: double, fastball_avg_spin: int, fastball_avg_break: double, breaking_avg_speed: double, breaking_avg_spin: int, breaking_avg_break: double, offspeed_avg_speed: double, offspeed_avg_spin: int, offspeed_avg_break: double, _c26: string]

In [56]:
df.limit(10).show()

+----------+----------+---------+----+-----------+------------+-----+---------------+--------------------+----------------+---------------------+------------------+----------------+------------------+---------------+------------+-------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+----+
| last_name|first_name|player_id|year|p_k_percent|p_bb_percent|xwoba|z_swing_percent|z_swing_miss_percent|oz_swing_percent|oz_swing_miss_percent|oz_contact_percent|out_zone_percent|iz_contact_percent|in_zone_percent|edge_percent|whiff_percent|fastball_avg_speed|fastball_avg_spin|fastball_avg_break|breaking_avg_speed|breaking_avg_spin|breaking_avg_break|offspeed_avg_speed|offspeed_avg_spin|offspeed_avg_break|_c26|
+----------+----------+---------+----+-----------+------------+-----+---------------+--------------------+----------------+---------------------+------------------+--

In [58]:
## SQL 사용이 가능할 수 있도록 titanic_sdf DataFrame을 DB View로 생성. 
df.createOrReplaceTempView('df_view')

In [13]:
spark.sql('select * from df_view').show()

+----------+-----------+---------+----+-----------+------------+-----+---------------+--------------------+----------------+---------------------+------------------+----------------+------------------+---------------+------------+-------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+----+
| last_name| first_name|player_id|year|p_k_percent|p_bb_percent|xwoba|z_swing_percent|z_swing_miss_percent|oz_swing_percent|oz_swing_miss_percent|oz_contact_percent|out_zone_percent|iz_contact_percent|in_zone_percent|edge_percent|whiff_percent|fastball_avg_speed|fastball_avg_spin|fastball_avg_break|breaking_avg_speed|breaking_avg_spin|breaking_avg_break|offspeed_avg_speed|offspeed_avg_spin|offspeed_avg_break|_c26|
+----------+-----------+---------+----+-----------+------------+-----+---------------+--------------------+----------------+---------------------+------------------

In [14]:
spark.sql('select * from df_view limit(10)')

DataFrame[last_name: string,  first_name: string, player_id: int, year: int, p_k_percent: double, p_bb_percent: double, xwoba: double, z_swing_percent: double, z_swing_miss_percent: double, oz_swing_percent: double, oz_swing_miss_percent: double, oz_contact_percent: double, out_zone_percent: double, iz_contact_percent: double, in_zone_percent: double, edge_percent: double, whiff_percent: double, fastball_avg_speed: double, fastball_avg_spin: int, fastball_avg_break: double, breaking_avg_speed: double, breaking_avg_spin: int, breaking_avg_break: double, offspeed_avg_speed: double, offspeed_avg_spin: int, offspeed_avg_break: double, _c26: string]

In [23]:
spark.sql('select upper(last_name),xwoba from df_view').show()

+----------------+-----+
|upper(last_name)|xwoba|
+----------------+-----+
|           PEREZ|0.274|
|      WAINWRIGHT|0.324|
|         GREINKE|0.299|
|           PETIT|0.328|
|         SANCHEZ|0.355|
|          RONDON|0.355|
|        KINTZLER|0.325|
|          JANSEN|0.266|
|          CHAVEZ|0.393|
|        DETWILER|0.294|
|            HILL|0.342|
|          GUERRA|0.321|
|          MORTON|0.305|
|          LESTER|0.363|
|          HUGHES|0.328|
|         KENNEDY|0.377|
|          MILLER| 0.22|
|          WATSON|0.303|
|            BARD|0.306|
|         LEBLANC|0.323|
+----------------+-----+
only showing top 20 rows



In [27]:
spark.sql('select substring(last_name, 0, 1) from df_view').show()

+--------------------------+
|substring(last_name, 0, 1)|
+--------------------------+
|                         P|
|                         W|
|                         G|
|                         P|
|                         S|
|                         R|
|                         K|
|                         J|
|                         C|
|                         D|
|                         H|
|                         G|
|                         M|
|                         L|
|                         H|
|                         K|
|                         M|
|                         W|
|                         B|
|                         L|
+--------------------------+
only showing top 20 rows



In [62]:
spark.sql("""select last_name,first_name,fastball_avg_speed,xwoba from df_view
where fastball_avg_speed < 90 and xwoba < 0.25
""").show()

+---------+----------+------------------+-----+
|last_name|first_name|fastball_avg_speed|xwoba|
+---------+----------+------------------+-----+
|Zastryzny|       Rob|              88.9|0.212|
|     Hart|    Donnie|              88.3|  0.2|
|    Smith|       Joe|              89.0|0.249|
|    Suter|     Brent|              87.5|0.223|
|   Rogers|     Tyler|              82.4|0.218|
|   McHugh|    Collin|              88.3|0.233|
|    O'Day|    Darren|              86.6|0.244|
|     Nuno|     Vidal|              89.0|0.249|
|   Cimber|      Adam|              86.5|0.248|
+---------+----------+------------------+-----+



In [0]:
%sql
select pclass, name, age from titanic_view where upper(name) like '%MR.%' 

In [0]:
%sql
select * from titanic_view where pclass in (1, 2) 

In [0]:
%sql
-- order by 절로 정렬. pclass로 오름 차순 정렬 후 10건만 추출. 
select * from titanic_view order by pclass limit 10 -- titanic_sdf.orderBy('pclass').limit(10).show()

In [0]:
%sql
-- pclass는 오름차순, name은 내림차순으로 정렬
select * from titanic_view order by pclass asc, name desc

In [0]:
%sql
-- view의 건수 추출 
select count(*) as cnt from titanic_view -- titanic_sdf.count()

In [0]:
%sql
-- titanic_view의 최대 age, 최소 age, 평균 sibsp 값을 추출. 
select max(age), min(age), avg(sibsp) from titanic_view 


In [0]:
%sql
-- pclass로 group by 하여 pclass 레벨로 최대 age, 최소 age, 평균 sibsp 값을 추출. 
select pclass, max(age), min(age), avg(sibsp) from titanic_view group by pclass

In [0]:
%sql
-- pclass로 group by 하여 pclass 레벨로 최대 age, 최소 age, 평균 sibsp 값을 추출. 
select pclass, max(age) as max_age, min(age) as min_age, avg(sibsp) as avg_sibsp from titanic_view group by pclass having max(age) > 70
/* 
import pyspark.sql.functions as F
titanic_sdf.groupBy('pclass').agg(F.max(col('age')).alias('max_age'), 
                                  F.min(col('age')).alis('min_age'), 
                                  F.avg(col('sibsp')).alias('avg_sibsp')).filter(col('max_age') > 70).show()
*/

In [0]:
%sql

select pclass, max_age, min_age, avg_sibsp
from ( 
      select pclass, max(age) as max_age, min(age) as min_age, avg(sibsp) as avg_sibsp from titanic_view group by pclass
) where max_age > 70

In [0]:
%sql
-- 아래 update sql은 동작하지 않음. 
update titanic_view set fare=fare*10 -- titanic_sdf.withColumn('Fare', col('Fare')*10)

In [0]:
%sql

delete from titanic_view;

In [0]:
%sql
-- Null 조건(Not Null 조건 )
select * from titanic_view where age is null and cabin is not null -- titanic_sdf.filter(F.col('age').isNull & F.col('cabin').isNotNull())

In [0]:
import pyspark.sql.functions as F

titanic_sdf.filter((F.col('age').isNull()) & (F.col('cabin').isNotNull())).show()

In [0]:
%sql

-- select case when 조건으로 if elif else 조건에 따라 값 생성. 
Select a.*, CASE WHEN age <= 5 THEN 'Baby'
                 WHEN age <=12 Then 'Child'
                 WHEN age <= 18 THEN 'Teenage'
                 WHEN age <= 25 THEN 'Student'
                 WHEN age <=35 THEN 'Young Adult'
                 WHEN age <=60 THEN 'Adult'
                 WHEN age IS NULL THEN 'NA'
                 ELSE 'Elderly' END as Age_Category from titanic_view a

In [0]:
titanic_sdf.withColumn('Age_category', F.when(F.col('Age') <= 5, 'Baby')
                                      .when(F.col('Age') <= 12, 'Child')
                                      .when(F.col('Age') <= 18, 'Teenage')
                                      .when(F.col('Age') <= 25, 'Student')
                                      .when(F.col('Age') <= 35, 'Young Adult')
                                      .when(F.col('Age') <= 60, 'Adult')
                                      .when(F.col('Age').isNull(), 'NA')
                                      .otherwise('Elderly')).show()

### Databricks의 내장된 Graph 시각화 기능 사용하기. 
* 일반적으로 spark의 경우 spark DataFrame을 pandas Dataframe으로 변환 후 matplotlib 이나 seaborn을 적용하여 시각화
* Spark에서 Pandas API와 매우 유사하게 DataFrame을 사용할 수 있도록 해주는 Koalas 패키지가 있음. Spark 3.2 부터는 Pandas API로 Koalas 패키지가 통합됨. 
* Databricks는 Graph 시각화 기능이 내장되어 있음. 이를 이용하여 손쉽게 시각화 가능. 
* 시각화 기능을 적용할 때에는 클라이언트 레벨에서 너무 많은 데이터를 가공하지 않도록 유의(매우 많은 시간과 리소스 소모)

In [0]:
%sql
--bar chart
select pclass, count(*) from titanic_view group by pclass

In [0]:
## SQL뿐만 아니라 DataFrame도 display() 적용 시 Databricks 시각화 기능 적용 가능. 
display(titanic_sdf.groupBy('Pclass').count())

In [0]:
%sql
-- histogram
select survived from titanic_view

In [0]:
%sql
-- bar chart
select Survived, count(*) from titanic_view group by Survived order by 2 desc

In [0]:
%sql 
-- Sex 별로 생존/사망 건수를 비교하여 보여 주기. 
select Survived, Sex, count(*) from titanic_view group by survived, sex;

In [0]:
%sql
-- pclass와 sex 별로 생존/사망 건수를 비교하여 보여 주기 
select Survived, Sex, Pclass, count(*) cnt from titanic_view group by survived, sex, pclass;

In [0]:
%sql
-- scatter plot
select fare, age from titanic_view

In [0]:
%sql
-- boxplot
select age from titanic_view